## Question 1

In [33]:
#Downloading the dependencies 

from bs4 import BeautifulSoup #library for web scraping
import requests  # library to handle requests
import json  # library to handle JSON files
import xml
import pandas as pd #Python library data manipulation and analysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
print("Imported Set 1 libraries")

Imported Set 1 libraries


In [31]:

from geopy.geocoders import Nominatim # convert an address into latitude and longitude values
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib as mpl
import matplotlib.cm as cm
import matplotlib.colors as colors
from matplotlib.patches import Polygon
from matplotlib.collections import PatchCollection
from matplotlib.colors import rgb2hex
import matplotlib.pyplot as plt
%matplotlib notebook
!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

# import k-means from clustering stage
from sklearn.cluster import KMeans
print("Imported Set 2 libraries")

Solving environment: done

## Package Plan ##

  environment location: /home/jupyterlab/conda

  added / updated specs: 
    - folium=0.5.0


The following packages will be UPDATED:

    ca-certificates: 2018.10.15-ha4d7672_0 conda-forge/label/gcc7 --> 2018.11.29-ha4d7672_0 conda-forge
    certifi:         2018.8.24-py36_1001   conda-forge/label/gcc7 --> 2018.11.29-py36_1000  conda-forge

The following packages will be DOWNGRADED:

    openssl:         1.0.2p-h14c3975_1001  conda-forge/label/gcc7 --> 1.0.2p-h470a237_1     conda-forge

Preparing transaction: done
Verifying transaction: done
Executing transaction: done
Imported Set 2 libraries


In [37]:
res = requests.get(" https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M")
soup = BeautifulSoup(res.content,'lxml')
table = soup.find_all('table')[0] 
df = pd.read_html(str(table))
print( tabulate(df[0], headers=['Postcode','Borough','Neighbourhod'], tablefmt='psql') )

+-----+------------+------------------+--------------------------------------------------+
|     | Postcode   | Borough          | Neighbourhod                                     |
|-----+------------+------------------+--------------------------------------------------|
|   0 | Postcode   | Borough          | Neighbourhood                                    |
|   1 | M1A        | Not assigned     | Not assigned                                     |
|   2 | M2A        | Not assigned     | Not assigned                                     |
|   3 | M3A        | North York       | Parkwoods                                        |
|   4 | M4A        | North York       | Victoria Village                                 |
|   5 | M5A        | Downtown Toronto | Harbourfront                                     |
|   6 | M5A        | Downtown Toronto | Regent Park                                      |
|   7 | M6A        | North York       | Lawrence Heights                                 |

In [38]:
# fetch Wikipedia HTML table
df = pd.read_html('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M', header=0)[0]

# filter unassigned boroughs
df = df[df['Borough'] != 'Not assigned']

# clean unassigned neighbourhoods
matcher = df['Neighbourhood'] == 'Not assigned'
df.loc[matcher] = df[matcher]['Borough']

# aggregate neighbourhoods for same postcode and borough
df = df.groupby(['Postcode', 'Borough'], as_index=False).agg(lambda neighborhoods: ', '.join(neighborhoods))

# rename column header
df.rename(columns={'Postcode':'PostalCode'}, inplace=True)

display(df.head(12))
df.shape

,PostalCode,Borough,Neighbourhood
0,M1B,Scarborough,"Rouge, Malvern"
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae
5,M1J,Scarborough,Scarborough Village
6,M1K,Scarborough,"East Birchmount Park, Ionview, Kennedy Park"
7,M1L,Scarborough,"Clairlea, Golden Mile, Oakridge"
8,M1M,Scarborough,"Cliffcrest, Cliffside, Scarborough Village West"
9,M1N,Scarborough,"Birch Cliff, Cliffside West"


(103, 3)

## Question 2

In [39]:
# fetch geo locations for postal codes
df_geo = pd.read_csv('http://cocl.us/Geospatial_data')

# rename column header
df_geo.rename(columns={'Postal Code':'PostalCode'}, inplace=True)

# add geo locations to main df
df_with_geo = df.merge(df_geo, on='PostalCode', how='left')
df_with_geo.head(12)

,PostalCode,Borough,Neighbourhood,Latitude,Longitude
0,M1B,Scarborough,"Rouge, Malvern",43.806686,-79.194353
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476
5,M1J,Scarborough,Scarborough Village,43.744734,-79.239476
6,M1K,Scarborough,"East Birchmount Park, Ionview, Kennedy Park",43.727929,-79.262029
7,M1L,Scarborough,"Clairlea, Golden Mile, Oakridge",43.711112,-79.284577
8,M1M,Scarborough,"Cliffcrest, Cliffside, Scarborough Village West",43.716316,-79.239476
9,M1N,Scarborough,"Birch Cliff, Cliffside West",43.692657,-79.264848
